In [42]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
import os

template = PromptTemplate.from_template("What is the distance between {country1} and {country2}?")

chat = ChatOpenAI(
  temperature=0.1,
  openai_api_key=os.getenv("OPEN_API_KEY"),
)

prompt = template.format(country1="Mexico", country2="Thailand")

chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles) when measured in a straight line.'

In [43]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
  SystemMessage(content="You are a geography expert. And you only reply in Italian"),
  AIMessage(content="Ciao, mi chiamo Paolo"),
  HumanMessage(content="What is the distance between the Mexico and Thailand? Also what is your name?"),
]

chat.predict_messages(messages)

AIMessage(content='Ciao! Il mio nome è Paolo. La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri.')

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import HumanMessage, AIMessage, SystemMessage

template = ChatPromptTemplate.from_messages(
    [
      ("system", "You are a geography expert. And you only reply in {language}."),
      ("ai", "Ciao, mi chiamo {name}!"),
      ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?"),
    ]
)

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand",
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα. Το όνομά μου είναι Σωκράτης. Πώς μπορώ να βοηθήσω;')

In [15]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
  def parse(self, text):
    items = text.strip().split(",")
    return list(map(str.strip, items))
  
  
p = CommaOutputParser()

p.parse("Hello, my name, is John")

['Hello', 'my name', 'is John']

In [34]:
template = ChatPromptTemplate.from_messages([
  ("system", "You are a list generating machine. Everthing you are asked will be answered with a comma separated list of max {max_items} in lowercase. DON'T reply with anything else."),
  ("human", "{question}"),
])
                                            
prompt = template.format_messages(
  max_items=10,
  question="What are the colors?",
)

result = chat.predict_messages(prompt)
p = CommaOutputParser()

p.parse(result.content)

red, blue, green, yellow, orange, purple, pink, black, white, brown

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [35]:
chain = template | chat | CommaOutputParser()

chain.invoke({
  "max_items": 5,
  "question": "What are the pokemons?",
})

pikachu, charmander, bulbasaur, squirtle, jigglypuff

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

In [40]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
import os

chat = ChatOpenAI(
  temperature = 0.1,
  streaming = True,
  callbacks=[StreamingStdOutCallbackHandler()],
  openai_api_key=os.getenv("OPEN_API_KEY")
)

chef_template = ChatPromptTemplate.from_messages([
  ("system", "You are a world-class international chef. You are asked to create a recipe with easy to find ingredients."),
  ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_template | chat

In [41]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it. "),
  ("human", "{recipe}"),
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
  "cuisine": "Indian",
})

Great choice! How about trying a classic Indian dish like Chicken Tikka Masala? It's a flavorful and aromatic dish that is sure to impress your friends and family. Here's a simple recipe for you to try:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1 tablespoon ginger, minced
- 1 can (14 oz) diced tomatoes
- 2 tablespoons tomato paste
- 1 tablespoon garam masala
- 1 teaspoon ground cumin
- 1 teaspoon ground coriander
- 1 teaspoon paprika
- 1/2 teaspoon turmeric
- 1/2 teaspoon cayenne pepper (adjust to taste)
- Salt and pepper to taste
- 1/2 cup heavy cream
- Fresh cilantro, chopped (for garnish)

Instructions:
1. In a bowl, mix together the yogurt, lemon juice, 1 tablespoon of vegetable oil, half of the minced garlic, half of the minced ginger, and a pinch of salt. Add the chicken pieces and marinate for at least 1 ho

AIMessageChunk(content="For a vegetarian version of Chicken Tikka Masala, you can replace the chicken with a plant-based alternative such as tofu or paneer. Here's how you can prepare these alternatives:\n\n1. Tofu: Use firm or extra-firm tofu for this recipe. Drain the tofu and press it to remove excess water. Cut the tofu into bite-sized cubes and follow the marinating instructions in the recipe. Tofu absorbs flavors well, so marinating it for a longer time will enhance the taste. Instead of baking, you can pan-fry the marinated tofu until it's golden brown and slightly crispy.\n\n2. Paneer: Paneer is a type of Indian cheese that holds its shape well when cooked. You can either make your own paneer at home or buy it from the store. Cut the paneer into bite-sized cubes and follow the marinating instructions in the recipe. Since paneer is already firm, you can skip the baking step and directly add the marinated paneer to the sauce in the skillet. Cook it gently to allow the flavors to 